In [ ]:
!pip install transformers langchain
!pip install langchain
!pip install pydantic==1.10.7

In [ ]:
!pip install --upgrade langchain transformers torch
!pip install pydantic==1.10.7

In [ ]:
!pip show langchain
!pip show pydantic
!pip show transformers
!pip show torch

In [25]:
from transformers import BertTokenizer, BertForQuestionAnswering, pipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
import torch

In [ ]:
# Specify the model name
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

In [27]:
# Initialize the QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

In [28]:
# Initialize the QA pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

In [29]:
# Wrap the pipeline with LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(
    pipeline=qa_pipeline,
    model_kwargs={"temperature": 0.7}
)

In [30]:
# Sample documents
documents = [
    "Hugging Face is creating a tool that democratizes AI.",
    "BERT stands for Bidirectional Encoder Representations from Transformers."
]

# Initialize embeddings using HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='bert-base-uncased')

# Create a FAISS vector store from the documents
vectorstore = FAISS.from_texts(documents, embeddings)

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [32]:
# Create a retriever from the vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})  # Retrieve top 1 relevant document

# Initialize the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" is a simple chain type that stuffs all retrieved docs into the prompt
    retriever=retriever,
    return_source_documents=True
)

ValidationError: 1 validation error for LLMChain
verbose
  Input should be a valid boolean [type=bool_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.9/v/bool_type